In [19]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
from matrix_factorization import matrix_factorization
from graph_init import *
from similarity import *
from create_R import *
from ALS import *
from hard_hfs import *
import copy
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
def RMSE(ground, predict):
    
    error = 0
    n = 0
    
    for i in range(len(ground)):
        for j in range(len(ground[0])):
            if ground[i,j] != 0:
                error += (ground[i,j] - predict[i,j])**2
                n += 1
                
    return np.sqrt(error/n)

def RMSEvec(ground, predict):
    
    error = 0
    n = 0
    
    for i in range(len(ground)):
        if ground[i] != 0:
            error += (ground[i] - predict[i])**2
            n += 1
                
    return np.sqrt(error/n)

def meanError(ground_truth,new_res):
    return np.mean(abs((new_res - ground_truth)[ground_truth!=0]))

In [21]:
def dictfromR(R):

    R_dict = {"Users": np.empty([0]), "Movies": np.empty([0]), "Ratings": np.empty([0])}

    for i in range(len(R)):
        for j in range(len(R[0])):
            if R[i,j] != 0:
                R_dict["Users"] = np.append(R_dict["Users"],i)
                R_dict["Movies"] = np.append(R_dict["Movies"],j)
                R_dict["Ratings"] = np.append(R_dict["Ratings"],R[i,j])

    return R_dict

In [22]:
# R = [
#      [5,3,0,1],
#      [4,0,0,1],
#      [1,1,0,5],
#      [1,0,0,4],
#      [0,1,5,4],
#     ]

# R = np.array(R)

R,R_dict = create_R()

print(R_dict)

/DATA/MVA/Graphes/Projet/cf-cold-start/src/create_R.py:21: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  R[ratingsnp[i,0]-1, ratingsnp[i,-1]] = ratingsnp[i,2]


{'Movies': array([   30.,   833.,   859., ...,  4597.,  4610.,  4696.]), 'Ratings': array([ 2.5,  3. ,  3. , ...,  4. ,  2.5,  3.5]), 'Users': array([   0.,    0.,    0., ...,  670.,  670.,  670.])}


In [23]:
P_dict = copy.deepcopy(R_dict)
P_dict["Ratings"] = np.ones([len(R_dict["Ratings"])])
P = R > 0
print(P)

[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [ True False False ..., False False False]
 [ True False False ..., False False False]]


In [24]:
np.argmax([np.sum(P[i,:] for i in range(len(P)))][0])

321

In [25]:
to_keep = 3
sel = np.where(R[:,321] != 0)[0]
np.random.shuffle(sel)
sel = sel[:len(sel)-to_keep]

In [26]:
ground_truth = copy.deepcopy(R[:,321])
R[sel,321] = 0

In [27]:
np.sum(R[:,321] != 0)

3

In [28]:
R_dictCopy = copy.deepcopy(R_dict)
R_dict = dictfromR(R)

In [29]:
np.where(R_dict['Movies'] == 321)

(array([32243, 38187, 60865]),)

In [30]:
N = len(R)
M = len(R[0])
K = 4

# P = np.random.rand(N,K)
# Q = np.random.rand(M,K)

# nP, nQ = matrix_factorization(R, P, Q, K)

als = ALS(K,N,M,"Users","Movies","Ratings",lbda = 0.1,lbda2 = 0.1)
print("Als created")
ans = als.fit(R_dict)

# nR = np.dot(nP, nQ.T)

# print(nP, "\n\n", nQ)

Als created


In [31]:
als.U

array([[-0.15242654, -0.45533908,  1.27029277,  0.25365663],
       [-0.69179104, -0.48861151,  1.44226795,  0.68412738],
       [ 0.01041743, -0.56259514,  1.63081172,  0.44446513],
       ..., 
       [-0.01325049, -0.1706424 ,  1.60993453,  0.94333564],
       [-0.89348623, -0.69766714,  1.10902946,  1.18137614],
       [-0.59727569, -0.04239626,  1.54349475,  0.82448632]])

In [32]:
als.V

array([[-0.52887893, -0.11881132,  1.74249985,  1.11987664],
       [-0.60910738, -0.17469848,  1.70938045,  0.40221901],
       [-0.86782057, -0.52988917,  1.43743075,  0.22129862],
       ..., 
       [-1.07349656,  0.66080902,  1.99662556,  0.14802337],
       [-0.64409794,  0.39648541,  1.19797534,  0.08881402],
       [ 0.61834456, -0.61556614,  1.95814377,  1.54350504]])

In [33]:
R_rec = np.dot(als.U,np.transpose(als.V))
print((R_rec-np.min(R_rec))*5/np.max(R_rec-np.min(R_rec)))
print(R_rec)

[[ 2.9336587   2.86700309  2.79900781 ...,  2.86369701  2.51523595
   3.08836627]
 [ 3.3165568   3.15354873  3.0951028  ...,  3.2083953   2.72205492
   3.33440232]
 [ 3.2084225   3.08601151  2.96917839 ...,  3.04331215  2.62300504
   3.50566815]
 ..., 
 [ 3.38298457  3.12566553  2.93100767 ...,  3.15649695  2.69091592
   3.674859  ]
 [ 3.35506274  3.07837698  3.06537227 ...,  3.02484819  2.61192666
   3.37692968]
 [ 3.39899251  3.18714479  3.04436951 ...,  3.35723148  2.81135663
   3.40541837]]
[[ 2.63226372  2.44583032  2.25564989 ...,  2.43658333  1.46195     3.06497536]
 [ 3.70321624  3.24728838  3.08381719 ...,  3.40069231  2.04041539
   3.75312952]
 [ 3.4007684   3.05838917  2.73161104 ...,  2.93896057  1.76337634
   4.23215404]
 ..., 
 [ 3.88901237  3.16930007  2.62484888 ...,  3.25553447  1.95332068
   4.7053749 ]
 [ 3.81091599  3.03703565  3.00066194 ...,  2.8873175   1.7323905
   3.87207705]
 [ 3.93378602  3.34125541  2.94191795 ...,  3.81698189  2.29018913
   3.95175894]]


In [34]:
print(RMSE(R,R_rec))
print(RMSE(R,(R_rec-np.min(R_rec))*5/np.max(R_rec-np.min(R_rec))))

0.700180433257
0.934839224769


In [35]:
print(RMSEvec(ground_truth, R_rec[:,321]))

0.97011494657


In [36]:
R

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 4.,  0.,  0., ...,  0.,  0.,  0.],
       [ 5.,  0.,  0., ...,  0.,  0.,  0.]])

In [37]:
np.max(R_rec)

8.4117477016733613

In [38]:
lp = LaplacianParams()

# sim = similarity(als.U)
sim = build_graph(als.U, GraphParams())
# Seems to work better with U... 

print(sim)

[[ 0.          0.          0.63628392 ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 ..., 
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]]


In [39]:
L = build_laplacian(sim,lp)

print(L.shape)

(671, 671)


In [40]:
supp = 100
test_vec = copy.deepcopy(R[:,321])*2
# test_vec[:supp] = [0 for i in range(supp)]
test_vec.shape

(671,)

In [41]:
# test_vec

In [42]:
hfs0, confidence = simple_hfs(als.U, test_vec, L, sim)
# hfs0/2

/home/marc/anaconda3/lib/python3.5/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)


In [43]:
maxconfidences = np.array([max(confidence[i,:]) for i in range(len(confidence))])

In [44]:
lim = np.percentile(maxconfidences, 1)
print(RMSEvec(ground_truth*(maxconfidences > lim),hfs0/2))
print(RMSEvec(ground_truth*(maxconfidences > lim), R_rec[:,321]))

0.994029797388
0.975188811326


In [45]:
print(meanError(ground_truth,hfs0/2))
print(meanError(ground_truth,R_rec[:,321]))

0.696480938416
0.743529222145


In [47]:
# elmnt = 321
# val = []
# print(RMSEvec(R[:,elmnt],R_rec[:,elmnt]))
# for supp in range(1,671,10):
#     test_vec = copy.deepcopy(R[:,elmnt])*2
#     test_vec[:supp] = [0 for i in range(supp)]

#     hfs0 = simple_hfs(als.U, test_vec, L, sim)
#     val.append(RMSEvec(R[:,elmnt],hfs0/2))
    
# plt.plot(range(1,671,10),val)

In [48]:
lhfs = []
lconf = []
for i in range(len(R[0])):
    if i%1000 == 0:
        print(i)
    hfs0, confidence = simple_hfs(als.U, R[:,i]*2, L, sim)
    maxconfidences = np.array([max(confidence[i,:]) for i in range(len(confidence))])
    
    lim = np.percentile(maxconfidences, 90)
    
    lhfs.append(hfs0/2)
    lconf.append(maxconfidences > lim)

R_barre = np.vstack(lhfs).T
confs = np.vstack(lconf).T
    
# R_barre[R_barre < 1] = .5
# R_barre[R_barre > 5] = 5

0


/home/marc/anaconda3/lib/python3.5/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)


1000
2000
3000
4000
5000
6000
7000
8000
9000


In [49]:
R_barre_limited = R_barre * confs

In [50]:
np.unique(R_barre_limited)

array([ 0. ,  0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ,  4.5,  5. ])

In [51]:
print(RMSE(R_barre_limited,R_rec))

0.547189870237


In [52]:
R_barre_final = copy.deepcopy(R_barre_limited)
R_barre_final[R != 0] = 0

In [53]:
R_dict_barre = dictfromR(R_barre_final)

In [59]:
N = len(R)
M = len(R[0])
K = 4

als_trans = ALS(K,N,M,"Users","Movies","Ratings",lbda = 0.1,lbda2 = 0.1)
print("Als created")
ans = als_trans.fitTransductive(R_dict,R_dict_barre,C1=1,C2=0.1)

Als created


In [60]:
R_rec_trans = np.dot(als_trans.U,np.transpose(als_trans.V))
print(RMSE(R_rec_trans,R_rec))

0.448602385625


In [62]:
print(RMSE(R,R_rec_trans))

0.714288642179
